In [187]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly
import seaborn as sns; sns.set()
from matplotlib import dates
import altair as alt
import math

In [188]:
#read the csv and show the head
jedi = pd.read_csv('jedi_v1_og.csv', low_memory=False)
jedi.head()

,Event #,GOES Flare Start Time,GOES Flare Peak Time,GOES Flare Class,Pre-Flare Start Time,Pre-Flare End Time,Flare Interrupt,9.4 Pre-Flare Irradiance [W/m2],13.1 Pre-Flare Irradiance [W/m2],13.3 Pre-Flare Irradiance [W/m2],...,103.2 by 63.0 Fitting Score,103.2 by 71.9 Fitting Score,103.2 by 72.2 Fitting Score,103.2 by 77.0 Fitting Score,103.2 by 79.0 Fitting Score,103.2 by 83.6 Fitting Score,103.2 by 95.0 Fitting Score,103.2 by 97.3 Fitting Score,103.2 by 97.7 Fitting Score,103.2 by 102.6 Fitting Score
0,1.0,2010-05-04 16:15:00.000,2010-05-04 16:29:00.000,C3.6,2010-05-04 08:29:00.000,2010-05-04 16:29:00.000,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,2010-05-05 07:09:00.000,2010-05-05 07:16:00.000,C2.3,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,True,0.000004,0.000002,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,2010-05-05 11:37:00.000,2010-05-05 11:52:00.000,C8.8,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,True,0.000004,0.000002,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,2010-05-05 17:13:00.000,2010-05-05 17:19:00.000,M1.2,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,False,0.000004,0.000002,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,2010-05-07 07:29:00.000,2010-05-07 07:42:00.000,C2.0,2010-05-06 23:42:00.000,2010-05-07 07:42:00.000,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
jedi_sub = jedi[['19.5 Depth [%]', '17.1 Depth [%]', '30.4 Depth [%]', '21.1 Depth [%]']]

In [190]:
def index_of_nans(values):
    """returns an array that contains the indices of when the elements equal nan, must pass it a 2d numpy array"""
    useless_euvs = []
    for index, elem in enumerate(values):
        if math.isnan(elem):
            useless_euvs.append(index)
    
    return useless_euvs

In [191]:
ind_of_empty_events = index_of_nans(jedi_sub.T.describe().values[1])

In [192]:
event_num = jedi.filter(regex='Event #')
GOES_flare_data= jedi.filter(regex='GOES Flare ')
pre_flare_start = jedi.filter(regex='Pre-Flare Start Time')
pre_flare_end = jedi.filter(regex='Pre-Flare End Time')
flare_interrupt = jedi.filter(regex='Flare Interrupt')

type(event_num)

pandas.core.frame.DataFrame

In [193]:
useless_event_nums = []
for i in range(len(ind_of_empty_events)):
    for k in range(len(jedi_sub)):
        if ind_of_empty_events[i] == k:
            useless_event_nums.append(event_num.values[k])

In [194]:
jedi_sub.head(10)

,19.5 Depth [%],17.1 Depth [%],30.4 Depth [%],21.1 Depth [%]
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,0.093239,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,0.420038,NaN,NaN
8,NaN,0.900706,0.054656,NaN
9,NaN,0.235694,0.105876,NaN


In [195]:
jedi_sub.describe()

,19.5 Depth [%],17.1 Depth [%],30.4 Depth [%],21.1 Depth [%]
count,343.000000,338.000000,621.000000,319.000000
mean,1.616278,1.557309,0.641082,1.639358
std,1.948313,1.525887,0.893133,2.086991
min,0.000333,0.002962,0.000526,0.006233
25%,0.376743,0.421045,0.130372,0.287005
50%,1.012425,1.096147,0.342509,0.824832
75%,2.062317,2.122782,0.714671,2.028294
max,13.024037,8.871504,6.777720,11.151263


In [196]:
#watch out if you run this more than once, end with the a weird recursion-ish problem
# DONT RUN AGAIN
frames = [event_num, GOES_flare_data, pre_flare_start, pre_flare_end, flare_interrupt, jedi_sub]
jedi_sub_v2 = pd.concat(frames, axis=1)


In [197]:
jedi_sub_copy = jedi_sub_v2.copy()

In [198]:
useless_event_nums

[array([1.]),
 array([2.]),
 array([3.]),
 array([5.]),
 array([6.]),
 array([7.]),
 array([12.]),
 array([13.]),
 array([14.]),
 array([15.]),
 array([16.]),
 array([17.]),
 array([22.]),
 array([25.]),
 array([26.]),
 array([35.]),
 array([37.]),
 array([37.]),
 array([39.]),
 array([42.]),
 array([43.]),
 array([44.]),
 array([46.]),
 array([52.]),
 array([52.]),
 array([52.]),
 array([52.]),
 array([53.]),
 array([53.]),
 array([55.]),
 array([58.]),
 array([58.]),
 array([59.]),
 array([59.]),
 array([62.]),
 array([64.]),
 array([65.]),
 array([66.]),
 array([67.]),
 array([68.]),
 array([69.]),
 array([72.]),
 array([72.]),
 array([73.]),
 array([73.]),
 array([74.]),
 array([74.]),
 array([75.]),
 array([76.]),
 array([77.]),
 array([83.]),
 array([83.]),
 array([85.]),
 array([91.]),
 array([95.]),
 array([96.]),
 array([97.]),
 array([101.]),
 array([102.]),
 array([103.]),
 array([104.]),
 array([105.]),
 array([106.]),
 array([107.]),
 array([108.]),
 array([112.]),
 array(

In [199]:
jedi_sub_copy

,Event #,GOES Flare Start Time,GOES Flare Peak Time,GOES Flare Class,Pre-Flare Start Time,Pre-Flare End Time,Flare Interrupt,19.5 Depth [%],17.1 Depth [%],30.4 Depth [%],21.1 Depth [%]
0,1.0,2010-05-04 16:15:00.000,2010-05-04 16:29:00.000,C3.6,2010-05-04 08:29:00.000,2010-05-04 16:29:00.000,True,NaN,NaN,NaN,NaN
1,2.0,2010-05-05 07:09:00.000,2010-05-05 07:16:00.000,C2.3,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,True,NaN,NaN,NaN,NaN
2,3.0,2010-05-05 11:37:00.000,2010-05-05 11:52:00.000,C8.8,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,True,NaN,NaN,NaN,NaN
3,4.0,2010-05-05 17:13:00.000,2010-05-05 17:19:00.000,M1.2,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,False,NaN,NaN,0.093239,NaN
4,5.0,2010-05-07 07:29:00.000,2010-05-07 07:42:00.000,C2.0,2010-05-06 23:42:00.000,2010-05-07 07:42:00.000,True,NaN,NaN,NaN,NaN
5,6.0,2010-05-08 04:51:00.000,2010-05-08 04:59:00.000,C9.3,2010-05-07 20:59:00.000,2010-05-08 04:59:00.000,True,NaN,NaN,NaN,NaN
6,7.0,2010-05-08 11:40:00.000,2010-05-08 11:50:00.000,C1.8,2010-05-07 20:59:00.000,2010-05-08 04:59:00.000,True,NaN,NaN,NaN,NaN
7,9.0,2010-06-12 00:30:00.000,2010-06-12 00:57:00.000,M2.0,2010-06-11 16:57:00.000,2010-06-12 00:57:00.000,True,NaN,0.420038,NaN,NaN
8,10.0,2010-06-12 03:57:00.000,2010-06-12 04:06:00.000,C1.0,2010-06-11 16:57:00.000,2010-06-12 00:57:00.000,True,NaN,0.900706,0.054656,NaN
9,11.0,2010-06-12 09:02:00.000,2010-06-12 09:17:00.000,C6.1,2010-06-11 16:57:00.000,2010-06-12 00:57:00.000,True,NaN,0.235694,0.105876,NaN


In [200]:
jedi_sub_noNaN_events = jedi_sub_copy.dropna(thresh=8)

In [204]:
jedi_sub_noNaN_events

,Event #,GOES Flare Start Time,GOES Flare Peak Time,GOES Flare Class,Pre-Flare Start Time,Pre-Flare End Time,Flare Interrupt,19.5 Depth [%],17.1 Depth [%],30.4 Depth [%],21.1 Depth [%]
3,4.0,2010-05-05 17:13:00.000,2010-05-05 17:19:00.000,M1.2,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,False,NaN,NaN,0.093239,NaN
7,9.0,2010-06-12 00:30:00.000,2010-06-12 00:57:00.000,M2.0,2010-06-11 16:57:00.000,2010-06-12 00:57:00.000,True,NaN,0.420038,NaN,NaN
8,10.0,2010-06-12 03:57:00.000,2010-06-12 04:06:00.000,C1.0,2010-06-11 16:57:00.000,2010-06-12 00:57:00.000,True,NaN,0.900706,0.054656,NaN
9,11.0,2010-06-12 09:02:00.000,2010-06-12 09:17:00.000,C6.1,2010-06-11 16:57:00.000,2010-06-12 00:57:00.000,True,NaN,0.235694,0.105876,NaN
16,18.0,2010-06-13 10:47:00.000,2010-06-13 10:53:00.000,C1.5,2010-06-12 21:39:00.000,2010-06-13 05:39:00.000,True,NaN,1.243839,0.121056,NaN
17,19.0,2010-06-14 00:44:00.000,2010-06-14 00:51:00.000,C1.5,2010-06-13 16:51:00.000,2010-06-14 00:51:00.000,False,NaN,NaN,0.005830,NaN
18,20.0,2010-07-08 22:03:00.000,2010-07-08 22:16:00.000,C1.2,2010-07-08 14:16:00.000,2010-07-08 22:16:00.000,True,NaN,NaN,0.332573,NaN
19,21.0,2010-07-09 08:53:00.000,2010-07-09 09:01:00.000,C1.8,2010-07-09 01:01:00.000,2010-07-09 09:01:00.000,True,NaN,0.570929,NaN,NaN
21,23.0,2010-07-13 10:43:00.000,2010-07-13 10:51:00.000,C2.6,2010-07-13 02:51:00.000,2010-07-13 10:51:00.000,False,NaN,1.259809,0.679178,NaN
22,24.0,2010-07-14 12:11:00.000,2010-07-14 12:30:00.000,C1.4,2010-07-14 04:30:00.000,2010-07-14 12:30:00.000,True,0.507776,NaN,0.038258,2.042699


In [203]:
jedi_sub_noNaN_events.filter(like='Depth').T.describe()

,3,7,8,9,16,17,18,19,21,22,...,4669,4676,4677,4682,4714,4719,4721,4722,4723,4736
count,1.000000,1.000000,2.000000,2.000000,2.000000,1.00000,1.000000,1.000000,2.000000,3.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000
mean,0.093239,0.420038,0.477681,0.170785,0.682448,0.00583,0.332573,0.570929,0.969494,0.862911,...,2.029286,1.679126,2.471866,3.070594,0.620604,0.325991,0.245076,0.971861,0.989749,0.338004
std,NaN,NaN,0.598248,0.091795,0.793928,NaN,NaN,NaN,0.410568,1.048349,...,NaN,NaN,NaN,NaN,0.367427,NaN,NaN,NaN,0.407547,NaN
min,0.093239,0.420038,0.054656,0.105876,0.121056,0.00583,0.332573,0.570929,0.679178,0.038258,...,2.029286,1.679126,2.471866,3.070594,0.166979,0.325991,0.245076,0.971861,0.701570,0.338004
25%,0.093239,0.420038,0.266169,0.138331,0.401752,0.00583,0.332573,0.570929,0.824336,0.273017,...,2.029286,1.679126,2.471866,3.070594,0.420892,0.325991,0.245076,0.971861,0.845660,0.338004
50%,0.093239,0.420038,0.477681,0.170785,0.682448,0.00583,0.332573,0.570929,0.969494,0.507776,...,2.029286,1.679126,2.471866,3.070594,0.650151,0.325991,0.245076,0.971861,0.989749,0.338004
75%,0.093239,0.420038,0.689194,0.203239,0.963144,0.00583,0.332573,0.570929,1.114652,1.275237,...,2.029286,1.679126,2.471866,3.070594,0.849863,0.325991,0.245076,0.971861,1.133839,0.338004
max,0.093239,0.420038,0.900706,0.235694,1.243839,0.00583,0.332573,0.570929,1.259809,2.042699,...,2.029286,1.679126,2.471866,3.070594,1.015135,0.325991,0.245076,0.971861,1.277929,0.338004


In [213]:
jedi_sub_noNaN_events.to_csv(path_or_buf='jedi_condensed.csv', index=False)